In [1]:
import torch

# Seleccionamos dispositivo CUDA si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cuda


In [9]:
import os, shutil, glob

src = r"C:\Users\juanj\Desktop\Reconocimineto-AnimalesDomesticos-CNN-Explicabilidad\data\images"
for filepath in glob.glob(os.path.join(src, "*.jpg")):
    breed = os.path.basename(filepath).split("_")[0]
    dst_dir = os.path.join(src, breed)
    os.makedirs(dst_dir, exist_ok=True)
    shutil.move(filepath, os.path.join(dst_dir, os.path.basename(filepath)))


In [17]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Transformaciones básicas
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225])
])

# Carga completa
dataset = datasets.ImageFolder(
    root=r"C:\Users\juanj\Desktop\Reconocimineto-AnimalesDomesticos-CNN-Explicabilidad\data\images",
    transform=transform
)
n = len(dataset)
n_train = int(n * 0.8)
n_val   = n - n_train

train_ds, val_ds = random_split(dataset, [n_train, n_val], 
                                generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=4, pin_memory=True)


In [18]:
num_classes = len(dataset.classes)  # ImageFolder guarda la lista de carpetas en .classes
print(f"Detectadas {num_classes} clases")

Detectadas 35 clases


In [19]:
import torch.nn as nn
from torchvision.models import resnet50

class FineTuneResNet50(nn.Module):
    def __init__(self, num_classes=37):
        super().__init__()
        self.backbone = resnet50(pretrained=True)
        # Congelamos todas las capas base
        for param in self.backbone.parameters():
            param.requires_grad = False

        # Reemplazamos la "fc" con nuestra cabeza:
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

model = FineTuneResNet50(num_classes=num_classes).to(device)


c:\Users\juanj\Desktop\Reconocimineto-AnimalesDomesticos-CNN-Explicabilidad\.venv2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\juanj\Desktop\Reconocimineto-AnimalesDomesticos-CNN-Explicabilidad\.venv2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.backbone.fc.parameters(), lr=1e-4)


In [21]:
# === CELDA: Entrenamiento y guardado de checkpoint para explicabilidad ===
import torch, os

# --- 1) Definir funciones de entrenamiento/evaluación (ya importaste model, train_loader, val_loader, criterion, optimizer, device) ---
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct = 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss   += loss.item() * images.size(0)
        total_correct+= (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader.dataset), total_correct / len(loader.dataset)

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss   += loss.item() * images.size(0)
            total_correct+= (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader.dataset), total_correct / len(loader.dataset)

# --- 2) Bucle de entrenamiento ---
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss,   val_acc   = evaluate(model,   val_loader,   criterion, optimizer, device)
    print(f"Epoch {epoch:02d} | "
          f"Train: {train_loss:.3f}, {train_acc:.2%} | "
          f"Val:   {val_loss:.3f}, {val_acc:.2%}")

# --- 3) Guardar checkpoint completo ---
ckpt = {
    "model_state_dict":     model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "num_classes":          num_classes,
    "class_to_idx":         dataset.class_to_idx,
    "transform": {
        "resize":            (224, 224),
        "normalize_mean":    [0.485, 0.456, 0.406],
        "normalize_std":     [0.229, 0.224, 0.225]
    },
    "split": {
        "seed":              42,
        "train_val_ratio":   0.8
    }
}
os.makedirs("checkpoints", exist_ok=True)
torch.save(ckpt, "checkpoints/resnet50_animals_exp.pth")
print("✅ Entrenamiento completado y checkpoint guardado en checkpoints/resnet50_animals_exp.pth")


TypeError: evaluate() takes 4 positional arguments but 5 were given

In [ ]:
# === CELDA: Guardar todo lo necesario para Captum ===
import torch, os, json

# Directorio de checkpoints
ckpt_dir = "checkpoints"
os.makedirs(ckpt_dir, exist_ok=True)
ckpt_path = os.path.join(ckpt_dir, "resnet50_animals_exp.pth")

# Recopilar metadata y estados
checkpoint = {
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "num_classes": num_classes,
    "class_to_idx": dataset.class_to_idx,
    "transform": {
        "resize": (224, 224),
        "normalize_mean": [0.485, 0.456, 0.406],
        "normalize_std":  [0.229, 0.224, 0.225]
    },
    "split_seed": 42,
    "train_val_ratio": 0.8
}

# Guardar checkpoint
torch.save(checkpoint, ckpt_path)
print(f"🔖 Checkpoint guardado en: {ckpt_path}")
